# 가사 web_crawling

## 장르

In [1]:
# 모듈 로딩
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd


year_song = list()  # 가사 담을 리스트
year_gene = list()  # 장르 담을 리스트
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}
for i in range(1970,2022):
  
    url= f'https://www.genie.co.kr/chart/musicHistory?year='+str(i)+'&category=0'
    html = requests.get(url,headers=headers)
    soup = BeautifulSoup(html.text, 'html.parser')
    div = soup.find('div', {'id':'wrap-body'})
    div_input = div.find('input', {'id':'sAllSongID'})
    div_input_value = div_input['value']
    url_list = div_input_value.split(';')  # 노래의 주소 담기

    for k in url_list[:-1]:  # 노래의 가사 담기
        url1 = f'https://www.genie.co.kr/detail/songInfo?xgnm='+str(k)
        html1 = requests.get(url1,headers=headers)
        soup1 = BeautifulSoup(html1.text, 'html.parser')
        div1 = soup1.find('div',{'class':'tit-box'})
        ul1 = soup1.find('ul',{'class':'info-data'})
        span2 = ul1.find_all('span',{'class':'value'})[2].text
        div_p1 = div1.find('p').text
        
        year_gene.append('/'.join(span2.split('/ ')[1:]))
        year_song.append(div_p1)


In [2]:
# 데이터프레임으로 만들기
df = pd.DataFrame([year_song, year_gene])
df = df.T

In [3]:
# 가사에서 띄어쓰기 제거
import re

for idx in df[0].index:
    df.iloc[idx, 0] = re.sub('\n|\r', ' ', df.iloc[idx, 0])

In [4]:
df.to_csv('gasa.csv', index=False)